In [ ]:
import sklearn
import numpy as np
import pandas as pd

In [ ]:
import cv2
import os
from time import time

In [ ]:
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
letter_number = {}
cont = 0
for letter in letters:
    letter_number[letter] = cont
    cont += 1
    
# el 9(J) y el 25(Z) están cogidos
# aunque no van a aparecer nunca voy a dejarlos con su neurona
# por lo que ahora tendré que poner 3 neuronas más de salida:
# la de la Z (aunque no se use, como la de J), la de SPACE y la de STOP
letter_number["_SPACE"] = 26
letter_number["_STOP"] = 27

letters_list = []
for letra in letters:
    letters_list.append(letra)
letters_list.append("_SPACE")
letters_list.append("_STOP")


number_letter = {}
for gakoa in letter_number:
    number_letter[letter_number.get(gakoa)] = gakoa

## In the following block, the used model is loaded. Depending on the folder where it has been placed, it may be necessary to change the path.

## En el siguiente bloque se carga el modelo utilizado, depende de la carpeta en la que se ha colocado es necesario cambiar el enlace

In [ ]:
modelo = tf.keras.models.load_model("final_model.h5")

### The following blocks are directly part of the program. The first one contains a function that is used in the program, and by executing the second block, the process starts.

### When executing the second block, the process starts and the computer's camera feed is displayed on three screens. One screen shows everything captured, with a blue square; another screen only shows what is inside the blue square, and a third screen is a small display that shows the content inside the blue square after resizing it and converting the color to grayscale.

### To start the program, it is necessary to select the screen that captures the entire image, titled 'Copy', and then hold down the 2 key on the keyboard (sometimes it takes a little while to respond). The process does not stop with the stop symbol; it is necessary to hold down the 2 key again.

### To exit the screens, simply hold down the 1 key or the q key.

### Los siguientes bloques son directamente parte del programa. El primero contiene una función que utiliza el programa y al ejecutar el segundo se pone en marcha el proceso.

### Al ejecutar el segundo se pone en marcha el proceso, aparece lo que capta la cámara del ordenador en tres pantallas (una dnde se ve todo lo que capta, con un cuadrado azul; otra dónde solo se ve lo que está dentro del cuadrado azul´y una última muy pequeña que enseña lo captado dentro del bloque azul después de reducir su tamaño y convertir el color a escala de grises).

### Para poner en marcha el programa es necesario seleccionar la pantalla que capta toda la imágen, la titulada como 'Copy', y después mantener pulsado el botón 2 del teclado (a veces le cuesta un poco). El proceso no se detiene con el símbolo de parada, es necesario volver a mantener pulsado el botón 2.

### Para salir de las pantallas basta con mantener pulsado la tecla 1 o la tecla q.

In [ ]:
def prediccion_y_confianza(imagen):
    
    # preprocesado con la imagen original
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = cv2.resize(imagen, (56, 56), interpolation = cv2.INTER_AREA)
    
    # preparación para predecir la imagen
    imagen = imagen.reshape(1, 56, 56, 1)
    imagen = imagen / 255
    
    # prediccion y confianza
    valores = modelo.predict(imagen)
    i_pred = np.argmax(valores[0])
    confianza = valores[0][i_pred]
    confianza = confianza * 100
    prediccion = number_letter[i_pred]
    
    return prediccion, confianza

In [ ]:
cap = cv2.VideoCapture(0)

t = time()
empieza = False

while cap.isOpened():
    
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    roi = frame[100 : 400, 320 : 620]

    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255, 0, 0), 5)

    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
    if cv2.waitKey(1) & 0xFF == ord("1"):
        break
        
    if cv2.waitKey(1) & 0xFF == ord("2"): empieza = True
    
    if empieza:
        
        #cv2.putText(copy, "ANALIZANDO", (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)
        
        if cv2.waitKey(1) & 0xFF == ord("3"): empieza = False
        
        now = time()
        dif = now - t
        
        if dif > 0.2:
            pred, conf = prediccion_y_confianza(roi)
            
            cv2.putText(copy, "pred: " + str(pred) + "   conf: " + str(round(conf, 2)) + "%",
                        (150, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)
            
    else: cv2.putText(copy, "EN PAUSA", (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)
 
    cv2.imshow("ROI", roi)
    cv2.imshow("Pantalla", copy)
    
    roi_copy = roi.copy()
    gray = cv2.cvtColor(roi_copy, cv2.COLOR_BGR2GRAY)
    gray_rs = cv2.resize(gray, (28, 28), interpolation = cv2.INTER_AREA)
    cv2.imshow("Procesado", gray_rs)


cap.release()
cv2.destroyAllWindows()  